In [32]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [33]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [34]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    input_index = np.array(range(0, length, 24))
    label_index = input_index + 24* (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), pred_day, labels

In [35]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"


In [36]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [37]:
file_parameters

{'watershed': 'yeong_auto'}

In [38]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [39]:
pd.set_option('display.max_columns', 1000)

In [40]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [41]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [42]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 200, True, 200)

In [43]:
run_num = range(len(folder))
real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)
#length = 1

In [44]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/yeong/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['나주_2015.xlsx', '나주_2016.xlsx', '나주_2017.xlsx', '나주_2018.xlsx', '나주_2019.xlsx', '나주_2020.xlsx'], ['지석천_2015.xlsx', '지석천_2016.xlsx', '지석천_2017.xlsx', '지석천_2018.xlsx', '지석천_2019.xlsx', '지석천_2020.xlsx'], ['용봉_2015.xlsx', '용봉_2016.xlsx', '용봉_2017.xlsx', '용봉_2018.xlsx', '용봉_2019.xlsx', '용봉_2020.xlsx'], ['우치_2015.xlsx', '우치_2016.xlsx', '우치_2017.xlsx', '우치_2018.xlsx', '우치_2019.xlsx', '우치_2020.xlsx']]
data/yeong/자동/나주_2015.xlsx
data/yeong/자동/나주_2016.xlsx
data/yeong/자동/나주_2017.xlsx
data/yeong/자동/나주_2018.xlsx
data/yeong/자동/나주_2019.xlsx
data/yeong/자동/나주_2020.xlsx
You must check this make_timeseries : time length
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/yeong/자동/지석천_2015.xlsx
data/yeong/자동/지석천_2016.xlsx
data/yeong/자동/지석천_2017.xlsx
data/yeong/자동/지석천_2018.xlsx
data/yeong/자동/지석천_2019.xlsx
data/yeong/자동/지석천_2020.xlsx
You must check this make_timeseries : time 

Epoch 47/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.3919 - disc_loss: 0.1019 - rmse: 0.1946 - val_loss: 0.2141
Epoch 48/200
595/595 [==============================] - 7s 11ms/step - gen_loss: 2.4323 - disc_loss: 0.1012 - rmse: 0.1909 - val_loss: 0.2203
Epoch 49/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.3876 - disc_loss: 0.1009 - rmse: 0.1947 - val_loss: 0.2165
Epoch 50/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.4165 - disc_loss: 0.1014 - rmse: 0.1956 - val_loss: 0.2097
Epoch 51/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.4197 - disc_loss: 0.1016 - rmse: 0.2043 - val_loss: 0.2122
Epoch 52/200
595/595 [==============================] - 7s 11ms/step - gen_loss: 2.4440 - disc_loss: 0.1015 - rmse: 0.1945 - val_loss: 0.2072
Epoch 53/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.3795 - disc_loss: 0.1003 - rmse: 0.1949 - val_loss: 0.2094
Epoch 

595/595 [==============================] - 7s 11ms/step - gen_loss: 2.4201 - disc_loss: 0.0981 - rmse: 0.1848 - val_loss: 0.2057
Epoch 105/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.4525 - disc_loss: 0.0978 - rmse: 0.1897 - val_loss: 0.2047
Epoch 106/200
595/595 [==============================] - 7s 11ms/step - gen_loss: 2.4072 - disc_loss: 0.0974 - rmse: 0.1955 - val_loss: 0.2045
Epoch 107/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.4116 - disc_loss: 0.0974 - rmse: 0.1855 - val_loss: 0.2078
Epoch 108/200
595/595 [==============================] - 6s 11ms/step - gen_loss: 2.4482 - disc_loss: 0.0974 - rmse: 0.1919 - val_loss: 0.2107
Epoch 109/200
595/595 [==============================] - 7s 11ms/step - gen_loss: 2.3846 - disc_loss: 0.0974 - rmse: 0.1981 - val_loss: 0.2143
Epoch 110/200
595/595 [==============================] - 7s 11ms/step - gen_loss: 2.4340 - disc_loss: 0.0989 - rmse: 0.1890 - val_loss: 0.2019
Epoch 111/200

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------

In [45]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  6
out_num_features :  1


In [57]:
print(real_df_all.shape)

rnn_target_column = 'do'
print(rnn_target_column)

(52608, 48)
do


In [58]:
#train_df, val_df, test_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, val=1, test=1'

train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
teg_check = 'train=8, test=1, val=1'

#val_df, test_df,train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'val=1, test=1, train=8'

#test_df, val_df, train_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.1, 0.8)
#teg_check = 'test=1, val=1, train=8'

#val_df, train_df, test_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'val=1, train=8, val=1'

#test_df, train_df, val_df, test_df2 = dataset_slice(real_df_all, 0.1, 0.8, 0.1)
#teg_check = 'test=1, train=8, val=1'


In [59]:
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (42086, 48) val_df.shape :  (5261, 48) test_df.shape: (5261, 48)


In [60]:
plt.figure()
plt.plot(real_df_all.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
plt.figure()
plt.plot(test_df.iloc[:, out_features[0]])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [62]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [63]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

target_col_idx :  2
out_num_features :  1


In [64]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}

In [65]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

model_path = "save/" + watershed + "models/" + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists('save/' + watershed):
        os.makedirs('save/' + watershed)
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/yeong/models/do/


## 모델 학습

In [66]:
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt")

Epoch 1/200
1/1 [==============================] - ETA: 0s - loss: 0.8891 - mean_absolute_error: 0.7517 - nse: -0.0554
Epoch 00001: val_loss improved from inf to 0.27572, saving model to save/best_model.h5
1/1 [==============================] - 1s 616ms/step - loss: 0.8891 - mean_absolute_error: 0.7517 - nse: -0.0554 - val_loss: 0.2757 - val_mean_absolute_error: 0.3573 - val_nse: -0.0863
Epoch 2/200
1/1 [==============================] - ETA: 0s - loss: 1.0599 - mean_absolute_error: 0.8570 - nse: 0.0175
Epoch 00002: val_loss improved from 0.27572 to 0.24857, saving model to save/best_model.h5
1/1 [==============================] - 0s 147ms/step - loss: 1.0599 - mean_absolute_error: 0.8570 - nse: 0.0175 - val_loss: 0.2486 - val_mean_absolute_error: 0.3434 - val_nse: -0.0979
Epoch 3/200
1/1 [==============================] - ETA: 0s - loss: 0.9636 - mean_absolute_error: 0.8113 - nse: 0.0742
Epoch 00003: val_loss improved from 0.24857 to 0.23151, saving model to save/best_model.h5
1/1 [==

Epoch 23/200
1/1 [==============================] - ETA: 0s - loss: 0.4158 - mean_absolute_error: 0.4947 - nse: 0.5188
Epoch 00023: val_loss did not improve from 0.22203
1/1 [==============================] - 0s 137ms/step - loss: 0.4158 - mean_absolute_error: 0.4947 - nse: 0.5188 - val_loss: 0.2525 - val_mean_absolute_error: 0.3630 - val_nse: 0.1783
Epoch 24/200
1/1 [==============================] - ETA: 0s - loss: 0.3306 - mean_absolute_error: 0.4385 - nse: 0.5442
Epoch 00024: val_loss improved from 0.22203 to 0.21651, saving model to save/best_model.h5
1/1 [==============================] - 0s 146ms/step - loss: 0.3306 - mean_absolute_error: 0.4385 - nse: 0.5442 - val_loss: 0.2165 - val_mean_absolute_error: 0.3406 - val_nse: 0.3207
Epoch 25/200
1/1 [==============================] - ETA: 0s - loss: 0.4862 - mean_absolute_error: 0.5209 - nse: 0.5645
Epoch 00025: val_loss did not improve from 0.21651
1/1 [==============================] - 0s 139ms/step - loss: 0.4862 - mean_absolute_

Epoch 46/200
1/1 [==============================] - ETA: 0s - loss: 0.3191 - mean_absolute_error: 0.4248 - nse: 0.6082
Epoch 00046: val_loss did not improve from 0.17529
1/1 [==============================] - 0s 140ms/step - loss: 0.3191 - mean_absolute_error: 0.4248 - nse: 0.6082 - val_loss: 0.1858 - val_mean_absolute_error: 0.2973 - val_nse: 0.3048
Epoch 47/200
1/1 [==============================] - ETA: 0s - loss: 0.3188 - mean_absolute_error: 0.4280 - nse: 0.6545
Epoch 00047: val_loss did not improve from 0.17529
1/1 [==============================] - 0s 139ms/step - loss: 0.3188 - mean_absolute_error: 0.4280 - nse: 0.6545 - val_loss: 0.2192 - val_mean_absolute_error: 0.3191 - val_nse: 0.2694
Epoch 48/200
1/1 [==============================] - ETA: 0s - loss: 0.3451 - mean_absolute_error: 0.4449 - nse: 0.5640
Epoch 00048: val_loss did not improve from 0.17529
1/1 [==============================] - 0s 140ms/step - loss: 0.3451 - mean_absolute_error: 0.4449 - nse: 0.5640 - val_loss: 

1/1 [==============================] - ETA: 0s - loss: 0.3183 - mean_absolute_error: 0.4320 - nse: 0.6502
Epoch 00069: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 140ms/step - loss: 0.3183 - mean_absolute_error: 0.4320 - nse: 0.6502 - val_loss: 0.2262 - val_mean_absolute_error: 0.3412 - val_nse: 0.2758
Epoch 70/200
1/1 [==============================] - ETA: 0s - loss: 0.3132 - mean_absolute_error: 0.4088 - nse: 0.6530
Epoch 00070: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.3132 - mean_absolute_error: 0.4088 - nse: 0.6530 - val_loss: 0.1738 - val_mean_absolute_error: 0.3018 - val_nse: 0.3908
Epoch 71/200
1/1 [==============================] - ETA: 0s - loss: 0.3046 - mean_absolute_error: 0.4152 - nse: 0.6290
Epoch 00071: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 138ms/step - loss: 0.3046 - mean_absolute_error: 0.4152 - nse: 0.6290 - val_loss: 0.1796 - val_

1/1 [==============================] - ETA: 0s - loss: 0.3326 - mean_absolute_error: 0.4256 - nse: 0.6643
Epoch 00092: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 140ms/step - loss: 0.3326 - mean_absolute_error: 0.4256 - nse: 0.6643 - val_loss: 0.1609 - val_mean_absolute_error: 0.2897 - val_nse: 0.3500
Epoch 93/200
1/1 [==============================] - ETA: 0s - loss: 0.3312 - mean_absolute_error: 0.4274 - nse: 0.6589
Epoch 00093: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 140ms/step - loss: 0.3312 - mean_absolute_error: 0.4274 - nse: 0.6589 - val_loss: 0.1959 - val_mean_absolute_error: 0.3199 - val_nse: 0.2899
Epoch 94/200
1/1 [==============================] - ETA: 0s - loss: 0.3072 - mean_absolute_error: 0.4132 - nse: 0.6822
Epoch 00094: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 141ms/step - loss: 0.3072 - mean_absolute_error: 0.4132 - nse: 0.6822 - val_loss: 0.1870 - val_

1/1 [==============================] - ETA: 0s - loss: 0.3153 - mean_absolute_error: 0.4112 - nse: 0.5809
Epoch 00115: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.3153 - mean_absolute_error: 0.4112 - nse: 0.5809 - val_loss: 0.2022 - val_mean_absolute_error: 0.3138 - val_nse: 0.3480
Epoch 116/200
1/1 [==============================] - ETA: 0s - loss: 0.2775 - mean_absolute_error: 0.4074 - nse: 0.7144
Epoch 00116: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.2775 - mean_absolute_error: 0.4074 - nse: 0.7144 - val_loss: 0.2162 - val_mean_absolute_error: 0.3381 - val_nse: 0.1804
Epoch 117/200
1/1 [==============================] - ETA: 0s - loss: 0.3275 - mean_absolute_error: 0.4228 - nse: 0.6703
Epoch 00117: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 140ms/step - loss: 0.3275 - mean_absolute_error: 0.4228 - nse: 0.6703 - val_loss: 0.1910 - va

1/1 [==============================] - ETA: 0s - loss: 0.3336 - mean_absolute_error: 0.4313 - nse: 0.6354
Epoch 00138: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.3336 - mean_absolute_error: 0.4313 - nse: 0.6354 - val_loss: 0.2816 - val_mean_absolute_error: 0.3913 - val_nse: 0.0151
Epoch 139/200
1/1 [==============================] - ETA: 0s - loss: 0.2842 - mean_absolute_error: 0.4058 - nse: 0.6781
Epoch 00139: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 138ms/step - loss: 0.2842 - mean_absolute_error: 0.4058 - nse: 0.6781 - val_loss: 0.2417 - val_mean_absolute_error: 0.3564 - val_nse: 0.2380
Epoch 140/200
1/1 [==============================] - ETA: 0s - loss: 0.2856 - mean_absolute_error: 0.4040 - nse: 0.5907
Epoch 00140: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 140ms/step - loss: 0.2856 - mean_absolute_error: 0.4040 - nse: 0.5907 - val_loss: 0.2527 - va

1/1 [==============================] - ETA: 0s - loss: 0.3366 - mean_absolute_error: 0.4243 - nse: 0.5336
Epoch 00161: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.3366 - mean_absolute_error: 0.4243 - nse: 0.5336 - val_loss: 0.2395 - val_mean_absolute_error: 0.3271 - val_nse: 0.3207
Epoch 162/200
1/1 [==============================] - ETA: 0s - loss: 0.3355 - mean_absolute_error: 0.4239 - nse: 0.6788
Epoch 00162: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 139ms/step - loss: 0.3355 - mean_absolute_error: 0.4239 - nse: 0.6788 - val_loss: 0.1910 - val_mean_absolute_error: 0.2997 - val_nse: 0.2876
Epoch 163/200
1/1 [==============================] - ETA: 0s - loss: 0.2985 - mean_absolute_error: 0.4011 - nse: 0.6613
Epoch 00163: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 138ms/step - loss: 0.2985 - mean_absolute_error: 0.4011 - nse: 0.6613 - val_loss: 0.2271 - va

1/1 [==============================] - ETA: 0s - loss: 0.2845 - mean_absolute_error: 0.3935 - nse: 0.7044
Epoch 00184: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 137ms/step - loss: 0.2845 - mean_absolute_error: 0.3935 - nse: 0.7044 - val_loss: 0.2880 - val_mean_absolute_error: 0.3895 - val_nse: -0.0535
Epoch 185/200
1/1 [==============================] - ETA: 0s - loss: 0.2439 - mean_absolute_error: 0.3685 - nse: 0.7871
Epoch 00185: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 138ms/step - loss: 0.2439 - mean_absolute_error: 0.3685 - nse: 0.7871 - val_loss: 0.2672 - val_mean_absolute_error: 0.3722 - val_nse: 0.0710
Epoch 186/200
1/1 [==============================] - ETA: 0s - loss: 0.3279 - mean_absolute_error: 0.4268 - nse: 0.6415
Epoch 00186: val_loss did not improve from 0.15424
1/1 [==============================] - 0s 138ms/step - loss: 0.3279 - mean_absolute_error: 0.4268 - nse: 0.6415 - val_loss: 0.2950 - v

## core / window.py / 

In [67]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  11.92
predict_day :  5
------------------------
1.5862554 4.465906 2.113269
nse3 :  [0.66854906]
pbias3 :  [9.149569]
save/yeong/models/do/
year : 2015 ~ 2020
run :  range(0, 1)
target :  do
train=8, test=1, val=1
---------------
[0.66854906]
[9.149569]


In [27]:
#train_df.shape, val_df.shape, test_df.shape

In [28]:
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------
#
#
#
#
#
#
#-----------------------------------------------------------------------------------------------------------------


In [69]:
from core.models import GRUModel, compile

In [71]:
model112 = GRUModel(rnn_out_steps, out_num_features)
model112.load_weights(model_path+"gru.ckpt")
compile(model112)


In [73]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=model112,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column)
print(teg_check)
print("---------------")
print(val_nse['GRU'])
print(val_pbias['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 :  11.92
predict_day :  5
------------------------
1.5862554 4.465906 2.113269
nse3 :  [0.66854906]
pbias3 :  [9.149569]
save/yeong/models/do/
year : 2015 ~ 2020
run :  range(0, 1)
target :  do
train=8, test=1, val=1
---------------
[0.66854906]
[9.149569]


In [75]:
model112.evaluate(multi_window.train)

ValueError: When providing an infinite dataset, you must specify the number of steps to run (if you did not intend to create an infinite dataset, make sure to not call `repeat()` on the dataset).

In [29]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    #print(df.shape)
    #print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
    

    predictions = model(inputs).numpy()
    
    print("input : ", inputs.shape, "labels : ", labels.shape, "predictions : ", predictions.shape)
    #print(predictions.shape)
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    print("                label_day.shape : ",label_day.shape, "pred_day.shape : ", pred_day.shape)
    
    #plt.figure()
    #input_index = np.array(range(0, length, 24))
    #label_index = input_index + 24* (7 + predict_day)
    #plt.plot(input_index, inputs_day[:, 0, :], label='input')
    #plt.plot(label_index, label_day[:, predict_day, :], label='label')
    #plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    #plt.legend()
    
    #o1 = np.mean(labels)
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('o1 : ', o1)
    print('predict_day : ', predict_day + 1)
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    print('------------------------')
    print(mae, mse ,rmse)
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    #print(labels_test.shape)
    #print(predis_test.shape)
    #print()
    #print("nse2_3 : ",nse2_3)
    #print("pbias2_3 : ",pbias2_3)
    ##print('------------------------')
    print("nse3 : ", nse3)
    print("pbias3 : ",pbias3)
    ##print('------------------------')
    
    return nse3, np.abs(pbias3), inputs_day, label_day, pred_day
    

In [30]:

x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '  ['+start_year+','+end_year+']  ' + rnn_target_column)
plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(x + width, val_nse.values(), 0.3, label='NSE')
plt.xticks(x,val_nse.keys(), rotation=0)
_ = plt.legend()
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TypeError: float() argument must be a string or a number, not 'NoneType'